In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

#### Import the Dataset

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


#### Split Data

In [6]:
x = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

In [7]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [8]:
print(y)

[1 0 1 ... 1 1 0]


#### Encoding Categorical Data

In [9]:
#### Encoding 'Gender' Column

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
x[:,2] = le.fit_transform(x[:,2])

In [10]:
x

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [13]:
#### ONE HOT ENCODING ON 'Geography' Column

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [14]:
x

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

### Split into training and test set

In [15]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

#### Feature Scaling

In [16]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [17]:
x_train

array([[ 1.00150113, -0.57946723, -0.57638802, ...,  0.64920267,
         0.97481699,  1.36766974],
       [-0.99850112,  1.72572313, -0.57638802, ...,  0.64920267,
         0.97481699,  1.6612541 ],
       [-0.99850112, -0.57946723,  1.73494238, ...,  0.64920267,
        -1.02583358, -0.25280688],
       ...,
       [ 1.00150113, -0.57946723, -0.57638802, ..., -1.54035103,
        -1.02583358, -0.1427649 ],
       [ 1.00150113, -0.57946723, -0.57638802, ...,  0.64920267,
        -1.02583358, -0.05082558],
       [-0.99850112,  1.72572313, -0.57638802, ...,  0.64920267,
         0.97481699, -0.81456811]])

In [18]:
x_test

array([[-1.02020406,  1.73668197, -0.56349184, ..., -1.57810057,
        -1.0502616 , -0.99096946],
       [ 0.98019606, -0.57581067, -0.56349184, ...,  0.63367318,
         0.95214374,  0.8293793 ],
       [-1.02020406, -0.57581067,  1.77464858, ...,  0.63367318,
        -1.0502616 , -0.69904548],
       ...,
       [ 0.98019606, -0.57581067, -0.56349184, ...,  0.63367318,
        -1.0502616 , -1.13742992],
       [ 0.98019606, -0.57581067, -0.56349184, ...,  0.63367318,
        -1.0502616 , -0.38237688],
       [-1.02020406,  1.73668197, -0.56349184, ...,  0.63367318,
         0.95214374,  0.15573964]])

## Building the ANN

#### Intializing the ANN

In [19]:
ann = tf.keras.models.Sequential()

#### Adding input layer and first Hidden layer

In [20]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

#### Adding Second hidden Layer

In [21]:
ann.add(tf.keras.layers.Dense(units=3,activation='relu'))

#### Adding output layer

In [22]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))  # sigmoid better probability

## Making predictions and evaluating the model

#### Compiling the ANN

In [23]:
ann.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

#### Training ANN on training set

In [24]:
ann.fit(x_train,y_train,batch_size=32,epochs=100)    # no.of preds(batch size) epochs for better accuracy

Epoch 1/100


250/250 [==============================] - 5s 4ms/step - loss: 0.6802 - accuracy: 0.6111
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4724 - accuracy: 0.7947
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4437 - accuracy: 0.8004
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4314 - accuracy: 0.8040
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4255 - accuracy: 0.8049
Epoch 6/100
250/250 [==============================] - 2s 6ms/step - loss: 0.4211 - accuracy: 0.8059
Epoch 7/100
250/250 [==============================] - 3s 12ms/step - loss: 0.4168 - accuracy: 0.8079
Epoch 8/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4103 - accuracy: 0.8095
Epoch 9/100
250/250 [==============================] - 2s 6ms/step - loss: 0.4019 - accuracy: 0.8144
Epoch 10/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3928 - acc

#### Predicting result of a single observation

In [26]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

1/1 [==============================] - 0s 35ms/step
[[False]]


###### The above prediction leads that the person has low percetage to leave the bank

#### Predicting Test results

In [28]:
y_pred = ann.predict(x_test)
y_pred = y_pred>0.5
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 3ms/step
[[0 0]
 [0 0]
 [0 0]
 ...
 [1 1]
 [0 1]
 [0 1]]


#### Confusion Matrix

In [31]:
from sklearn.metrics import confusion_matrix,accuracy_score

cm = confusion_matrix(y_test,y_pred)
cm
accuracy_score(y_test,y_pred)

0.857

In [32]:
!pip install tenserflow == 2.7.0

ERROR: Invalid requirement: '=='
